In [1]:
import pandas as pd
import requests
import time

In [2]:
session = requests.Session()

In [3]:
legislatures = [57, 56, 55, 54, 53, 52]

In [4]:
# for legislature in legislatures:
    
#     # request data from the Chamber of Deputies API
#     url = f'https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura={legislature}'
#     response = requests.get(url)
    
#     # parse json
#     data = response.json()
#     deputados = data['dados']
#     df = pd.DataFrame(deputados)
    
#     # get only unique deputies (changes in the same legislature, such as party or name, are not considered)
#     df.drop_duplicates(subset='id', inplace=True)
    
#     # save data
#     df.to_csv(f'data/deputies/deputies_{legislature}.csv', index=False)
    
#     # print number of (unique) saved deputies
#     print(f' Legislature {legislature}: {len(df)} saved deputies')

In [5]:
for legislature in legislatures:
    
    # deputies data
    df = pd.read_csv(f'data/deputies/deputies_{legislature}.csv')
    ids = set(df['id'].tolist())
    
    df = pd.DataFrame()
    
    # request data from the Chamber of Deputies API
    for id_ in ids:
        # instantiate page counter
        page = 1
        
        while True:
            # request data from the Chamber of Deputies API
            url = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id_}/discursos?idLegislatura={legislature}&pagina={page}&itens=100'
            response = session.get(url)
            
            if not response.ok:
                # if request fails, print error message, break loop and go to the next deputy
                print(f'Error: {response.text} - ID: {id_} - Page: {page}')
                break
            
            # parse json
            data = response.json()['dados']
            
            # if there is no data in page {page}, break loop and go to the next deputy
            if len(data) == 0:
                break
            
            # concatenate data
            df = pd.concat([df, pd.DataFrame(data)])
            print(f'Legislature {legislature}: Deputy {id_} - Page {page} - {len(data)} speeches')
            
            # save data every 100 speeches
            if len(df) % 100:
                df.to_csv(f'data/speeches/speeches_{legislature}.csv', index=False)
            
            # increment page counter
            page += 1
            
            time.sleep(5)
        
        # add deputy id to the dataframe
        df['id'] = id
    
    # save data for the current legislature
    df.to_csv(f'data/speeches/speeches_{legislature}.csv', index=False)

Error: upstream request timeout - ID: 215043 - Page: 1
Error: upstream request timeout - ID: 139285 - Page: 1


KeyboardInterrupt: 

In [4]:
session.close()